# Necessary Imports

In [1]:
pip install transformers torch onnx openvino-dev[onnx] datasets accelerate openvino peft bitsandbytes trl -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [2]:
!pip show transformers
!pip show accelerate

Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl
ERROR: Operation cancelled by user


In [1]:
!pip uninstall -y transformers accelerate
!pip install transformers[torch] accelerate

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)


In [2]:
import pandas as pd
import torch
from datasets import Dataset
from random import randrange
#from peft import Loraconfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer

# Load and Save Pre-trained Model and Tokenizer

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

model.save_pretrained("./my_t5_model")
tokenizer.save_pretrained("./my_t5_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

('./my_t5_model/tokenizer_config.json',
 './my_t5_model/special_tokens_map.json',
 './my_t5_model/spiece.model',
 './my_t5_model/added_tokens.json',
 './my_t5_model/tokenizer.json')

# Convert PyTorch Model to ONNX Format

In [17]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./my_t5_model")
model = AutoModelForSeq2SeqLM.from_pretrained("./my_t5_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        decoder_input_ids = torch.zeros((input_ids.shape[0], 1), dtype=torch.long)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            return_dict=False
        )
        return outputs[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "t5_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=14,
                  do_constant_folding=True)

print("Model converted to ONNX format successfully.")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1092: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


Model converted to ONNX format successfully.


# Convert ONNX Model to OpenVINO IR Format

In [18]:
import openvino as ov
import os

ir_model = ov.convert_model("t5_model.onnx",
                            input=[ov.PartialShape([1, 4])])

ov.save_model(ir_model, "t5_ir_model.xml")

print("Model converted to IR format successfully.")

print("XML file exists:", os.path.exists("t5_ir_model.xml"))
print("BIN file exists:", os.path.exists("t5_ir_model.bin"))

Model converted to IR format successfully.
XML file exists: True
BIN file exists: True


# Pre Processing Dataset

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/medquad.csv')

df = df[['question', 'answer']]

In [20]:
train_val_df, sample_df = train_test_split(df, test_size=0.1, random_state=42) #using only sample df as the whole dataset is very big

In [21]:
sample_df['text'] = 'Question:\n' + sample_df['question'] + '\n\nAnswer:\n' + sample_df['answer']
sample_df.drop(columns=['question','answer'], axis=1,inplace=True)
# sample_df.to_csv('/content/pre_dataset.csv', index=False)

In [22]:
df = pd.read_csv('/content/pre_dataset.csv')

train = Dataset.from_pandas(df)
train_dataset_size = len(train)
print(train_dataset_size)

1642


In [23]:
train

Dataset({
    features: ['text'],
    num_rows: 1642
})

# Load and Prepare Model, Tokenizer for finetuning

In [24]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset, DatasetDict
import torch

model_name = "./my_t5_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

train_dataset_size = len(train)
batch_size = 4
steps_per_epoch = train_dataset_size // batch_size

target_steps = 300
num_epochs = max(3, target_steps // steps_per_epoch)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
)

class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs is not None:
            if 'loss' in logs:
                print(f"Step {state.global_step}: Training Loss: {logs['loss']:.5f}")
            else:
                print(f"Step {state.global_step}: Logs: {logs}")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    dataset_text_field='text',
    max_seq_length=1042,
    tokenizer=tokenizer,
    packing=True,
    callbacks=[PrinterCallback()]
)

trainer.train()

trainer.save_model("./fine_tuned_t5_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will overri

Step,Training Loss
10,14.135000
20,13.908100
30,13.378400
40,11.947900
50,10.779400
60,8.887400
70,7.322400
80,5.418000
90,4.684900
100,3.570800


Step 10: Training Loss: 14.13500
Step 20: Training Loss: 13.90810
Step 30: Training Loss: 13.37840
Step 40: Training Loss: 11.94790
Step 50: Training Loss: 10.77940
Step 60: Training Loss: 8.88740
Step 70: Training Loss: 7.32240
Step 80: Training Loss: 5.41800
Step 90: Training Loss: 4.68490
Step 100: Training Loss: 3.57080
Step 110: Training Loss: 3.31060
Step 120: Training Loss: 3.05330
Step 130: Training Loss: 2.66950
Step 140: Training Loss: 2.38650
Step 150: Training Loss: 2.23420
Step 160: Training Loss: 2.01560
Step 170: Training Loss: 1.72260
Step 180: Training Loss: 1.61520
Step 190: Training Loss: 1.33180
Step 200: Training Loss: 1.25050
Step 210: Training Loss: 1.15370
Step 220: Training Loss: 1.08540
Step 230: Training Loss: 0.93390
Step 240: Training Loss: 0.90890
Step 250: Training Loss: 0.80610
Step 260: Training Loss: 0.78880
Step 270: Training Loss: 0.71990
Step 280: Training Loss: 0.69560
Step 290: Training Loss: 0.60700
Step 300: Training Loss: 0.59890
Step 310: Trai

# Exporting the Fine-Tuned GPT-Neo Model to ONNX Format

In [27]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_t5_model")
model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_t5_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        decoder_input_ids = torch.zeros((input_ids.shape[0], 1), dtype=torch.long)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            return_dict=False
        )
        return outputs[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "fine_tuned_t5_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11,
                  do_constant_folding=True)

print("Fine-tuned model converted to ONNX format successfully.")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1092: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


Fine-tuned model converted to ONNX format successfully.


/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:1738: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Min, node name: /model/encoder/block.0/layer.0/SelfAttention/Min): data_0 typestr: T, has unsupported type: tensor(int64) (Triggered internally at ../torch/csrc/jit/serialization/export.cpp:1469.)
  _C._check_onnx_proto(proto)


# Converting the Fine-Tuned GPT-Neo Model to OpenVINO IR Format

In [28]:
from openvino.tools import mo

ov_model = mo.convert_model("fine_tuned_t5_model.onnx",
                            input=[('input_ids', [1, -1]), ('attention_mask', [1, -1])],
                            output=['output'])

from openvino.runtime import serialize
serialize(ov_model, "fine_tuned_t5_ir_model.xml")

print("Fine-tuned model converted to IR format successfully.")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Fine-tuned model converted to IR format successfully.


# OpenVINO IR Format Fine Tuned Model Inference

In [29]:
import openvino.runtime as ov
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_t5_model")

core = ov.Core()
model = core.read_model("fine_tuned_t5_ir_model.xml")
compiled_model = core.compile_model(model, "CPU")

infer_request = compiled_model.create_infer_request()

def generate_text(prompt, max_length=200): # The maximum length is 200; the answer may be short.
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.numpy()
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.numpy()

    for _ in range(max_length):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}

        results = infer_request.infer(inputs)
        logits = results[next(iter(results))]

        next_token_id = np.argmax(logits[0, -1, :])

        input_ids = np.concatenate([input_ids, np.array([[next_token_id]])], axis=1)
        attention_mask = np.concatenate([attention_mask, np.array([[1]])], axis=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

prompt = "What is (are) Schwannomatosis ?"
generated_text = generate_text(prompt)
print("Generated text:", generated_text)

#True answer: Schwannomatosis is a rare form of neurofibromatosis that is primarily characterized by multiple schwannomas (benign tumors of the nervous system) in the absence of bilateral (affecting both sides) vestibular schwannomas. Signs and symptoms of the condition vary based on the size, location and number of schwannomas but may include pain; numbness; tingling; and/or weakness in the fingers and toes. Inherited forms of the disorder account for only 15 percent of all cases. In some of these families, schwannomatosis is caused by changes (mutations) in the SMARCB1 or LZTR1 genes; in other cases, the exact underlying cause is unknown. When inherited, the condition is passed down in an autosomal dominant manner with highly variable expressivity and reduced penetrance. Treatment is based on the signs and symptoms present in each person but may include medications and/or surgery.

Generated text: What is (are) Schwannomatosis? Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was Was


# Simple Fine Tuned Model Inference

In [31]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

fine_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_t5_model")
fine_model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_t5_model")

In [32]:
fine_model.cuda() # Using GPU for generating text

prompt = "What are the symptoms of Crome syndrome ?"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inputs = fine_tokenizer(prompt, return_tensors='pt', truncation=True, padding=True)
input_ids = inputs.input_ids.to(device)
attention_mask = inputs.attention_mask.to(device)

outputs = fine_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=200,
    temperature=0.6,
    pad_token_id=fine_tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [33]:
generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

What are the symptoms of Crome syndrome?


In [34]:
import time
from sklearn.metrics import accuracy_score
import psutil
import torch
from nltk.translate.bleu_score import sentence_bleu

# 1. Inference Time (Latency)
start_time = time.time()
generated_text = generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
end_time = time.time()
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.4f} seconds")

Inference Time: 0.0005 seconds


In [35]:
# 2. Throughput
num_samples = 10
start_time = time.time()
for _ in range(num_samples):
    generated_text = generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
end_time = time.time()
throughput = num_samples / (end_time - start_time)
print(f"Throughput: {throughput:.4f} samples/second")

Throughput: 5296.5071 samples/second


In [36]:
# 3. Model Load Time
start_time = time.time()
core = ov.Core()
model = core.read_model("fine_tuned_t5_ir_model.xml")
compiled_model = core.compile_model(model, "CPU")
end_time = time.time()
model_load_time = end_time - start_time
print(f"Model Load Time: {model_load_time:.4f} seconds")

Model Load Time: 0.6001 seconds


In [37]:
# 4. CPU Utilization
cpu_percent = psutil.cpu_percent(interval=1)  # Check CPU utilization over 1 second
print(f"CPU Utilization: {cpu_percent}%")

CPU Utilization: 4.5%


In [38]:
# 5. BLEU Score
question = "What are the symptoms of Crome syndrome ?"
answer = """What are the signs and symptoms of Crome syndrome? The Human Phenotype Ontology provides the following list of signs and symptoms for Crome syndrome.
If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to look up the definitions for these medical terms. Signs and Symptoms Approximate number of patients (when available) Abnormality of the renal tubule 90% Aplasia/Hypoplasia of the cerebellum 90% Cataract 90% Cognitive impairment 90% Encephalitis 90% Seizures 90% Nystagmus 50% Acute tubular necrosis -
Autosomal recessive inheritance - Cerebellar dysplasia - Congenital cataract - Encephalopathy - Intellectual disability - Short stature - The Human Phenotype Ontology (HPO) has collected information on how often a sign or symptom occurs in a condition. Much of this information comes from Orphanet, a European rare disease database.
 The frequency of a sign or symptom is usually listed as a rough estimate of the percentage of patients who have that feature. The frequency may also be listed as a fraction.
 The first number of the fraction is how many people had the symptom, and the second number is the total number of people who were examined in one study. For example, a frequency of 25/25 means that in a study of 25 people all patients were found to have that symptom.
 Because these frequencies are based on a specific study, the fractions may be different if another group of patients are examined. Sometimes, no information on frequency is available. In these cases, the sign or symptom may be rare or common."""

def generate_text_openvino(prompt, max_length=200):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.numpy()
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.numpy()

    for _ in range(max_length):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
        results = infer_request.infer(inputs)
        logits = results[next(iter(results))]
        next_token_id = np.argmax(logits[0, -1, :])

        input_ids = np.concatenate([input_ids, np.array([[next_token_id]])], axis=1)
        attention_mask = np.concatenate([attention_mask, np.array([[1]])], axis=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

generated_text = generate_text_openvino(question)
reference = [answer.split()]
candidate = generated_text.split()
bleu_score = sentence_bleu(reference, candidate)
print(f"BLEU Score: {bleu_score}")


BLEU Score: 0.011657608538801668


In [39]:
# 6. Fine-tuning Time
start_time = time.time()
trainer.train()
end_time = time.time()
fine_tuning_time = end_time - start_time
print(f"Fine-tuning Time: {fine_tuning_time:.4f} seconds")

Step,Training Loss
10,0.191200
20,0.210000
30,0.229300
40,0.207500
50,0.206900
60,0.210700
70,0.187300
80,0.200400
90,0.225500
100,0.189100


Step 10: Training Loss: 0.19120
Step 20: Training Loss: 0.21000
Step 30: Training Loss: 0.22930
Step 40: Training Loss: 0.20750
Step 50: Training Loss: 0.20690
Step 60: Training Loss: 0.21070
Step 70: Training Loss: 0.18730
Step 80: Training Loss: 0.20040
Step 90: Training Loss: 0.22550
Step 100: Training Loss: 0.18910
Step 110: Training Loss: 0.18600
Step 120: Training Loss: 0.23380
Step 130: Training Loss: 0.20340
Step 140: Training Loss: 0.18330
Step 150: Training Loss: 0.21070
Step 160: Training Loss: 0.17780
Step 170: Training Loss: 0.18590
Step 180: Training Loss: 0.18340
Step 190: Training Loss: 0.17500
Step 200: Training Loss: 0.21170
Step 210: Training Loss: 0.20170
Step 220: Training Loss: 0.19500
Step 230: Training Loss: 0.17240
Step 240: Training Loss: 0.17720
Step 250: Training Loss: 0.19150
Step 260: Training Loss: 0.20150
Step 270: Training Loss: 0.19800
Step 280: Training Loss: 0.19720
Step 290: Training Loss: 0.15480
Step 300: Training Loss: 0.16760
Step 310: Training 

In [40]:
metrics = {
    "Inference Time (Latency)": inference_time,
    "Throughput": throughput,
    "Model Load Time": model_load_time,
    "CPU Utilization": cpu_percent,
    "BLEU Score": bleu_score,
    "Fine-tuning Time": fine_tuning_time
}

with open("T5 metrics.txt", "w") as f:
    for key, value in metrics.items():
        f.write(f"{key}: {value}\n")

print("Metrics saved to metrics.txt")

Metrics saved to metrics.txt
